In [97]:
import torch
from torch import nn
from d2l import torch as d2l
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [34]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K),corr2d_multi_in(X, K).shape,X.shape,K.shape

(tensor([[ 56.,  72.],
         [104., 120.]]),
 torch.Size([2, 2]),
 torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]))

In [35]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [36]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [38]:
corr2d_multi_in_out(X, K),corr2d_multi_in_out(X, K).shape

(tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]),
 torch.Size([3, 2, 2]))

In [82]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0,1, (3, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
Y1,Y2

(tensor([[[-0.4488,  0.7491,  0.6433],
          [-2.4091, -1.0237, -0.1602],
          [ 0.1447,  1.0165,  1.0315]],
 
         [[ 0.7799,  1.2521,  1.5112],
          [-1.1390, -3.6220,  0.6093],
          [ 2.0472,  1.8107,  1.4237]],
 
         [[-0.0938,  1.9209,  1.2892],
          [-3.1382, -3.5727,  0.9126],
          [ 0.8266,  2.5288,  1.9770]]]),
 tensor([[[-0.4488,  0.7491,  0.6433],
          [-2.4091, -1.0237, -0.1602],
          [ 0.1447,  1.0165,  1.0315]],
 
         [[ 0.7799,  1.2521,  1.5112],
          [-1.1390, -3.6220,  0.6093],
          [ 2.0472,  1.8107,  1.4237]],
 
         [[-0.0938,  1.9209,  1.2892],
          [-3.1382, -3.5727,  0.9126],
          [ 0.8266,  2.5288,  1.9770]]]))

In [167]:
X = torch.normal(0, 1, (3,4,4))
# K = torch.normal(0, 1, (3,2,1))
# corr2d_multi_in(X,K).shape
# X.shape,K.shape
# corr2d_multi_in(X,K).shape
net = nn.Conv2d(3, 4, kernel_size=(2,1),padding=1)
net.weight.data.shape,net(X)# 4,5,6

(torch.Size([4, 3, 2, 1]),
 tensor([[[-0.3891, -1.7671, -1.6038, -0.1090, -0.8619, -0.3891],
          [-0.3891, -0.2639,  0.8497, -0.1410,  0.6053, -0.3891],
          [-0.3891,  0.1460, -0.3940, -1.3911, -0.7921, -0.3891],
          [-0.3891,  0.2739, -1.1849, -0.2155, -0.9967, -0.3891],
          [-0.3891, -0.8868,  0.2490, -0.1823,  0.0262, -0.3891]],
 
         [[ 0.3456,  2.0131, -0.0118,  0.3471,  0.6734,  0.3456],
          [ 0.3456, -0.7659, -0.6611,  0.1644,  0.7695,  0.3456],
          [ 0.3456, -0.6582,  0.7968, -0.2714, -0.5128,  0.3456],
          [ 0.3456,  0.2111,  0.3494,  0.1117, -0.9998,  0.3456],
          [ 0.3456,  0.6634, -0.0091,  0.2521,  0.4425,  0.3456]],
 
         [[-0.2749, -0.6208, -0.6701,  0.0252, -0.3993, -0.2749],
          [-0.2749, -0.1778, -0.6385, -0.3956,  0.6377, -0.2749],
          [-0.2749, -0.5977, -0.3783, -0.7059, -0.1648, -0.2749],
          [-0.2749, -0.6650, -0.3319, -1.0878, -1.6519, -0.2749],
          [-0.2749, -0.2866, -0.4792, -0.51

In [112]:
X = torch.normal(0, 1, (3,4,4))
K = torch.normal(0, 1, (2,3,2,2))
corr2d_multi_in(X,K)

ValueError: too many values to unpack (expected 2)

In [115]:
from torch import nn
from torchvision import models
model = models.resnet152()
model.fc = nn.Sequential(nn.Linear(2048, 2))
model.add_module('fn1', nn.Sequential(nn.Linear(2048, 2)))
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 